In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/arenda-zdaniy?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

41


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 2/2 [00:00<00:00,  2.76it/s]

41


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Аренда зданий'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 41/41 [00:09<00:00,  4.14it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Этажность,Материал стен,Объект,Назначение,Торг уместен,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/arenda-zdaniy/s-t-luch-...,"Аренда здания на с/т Луч терПриокский район, Н...",227 500 руб./мес.,Сдается в аренду помещение. \nЗдание находится...,2850812,Приокский,с/т Луч тер,455 м²,1,Уточнить,отдельно стоящее здание,"торговое, отдых и развлечения",NaN,gipernn,Комерческая недвижимость,Аренда зданий,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/arenda-zdaniy/ul-50-let...,Аренда здания на ул. 50-летия ПобедыМосковский...,240 000 руб./мес.,Сдается в аренду помещение свободного назначен...,2798090,Московский,ул. 50-летия Победы,400 м²,2,кирпич,отдельно стоящее здание,"офисное, медицинские учреждения",да,gipernn,Комерческая недвижимость,Аренда зданий,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/arenda-zdaniy/ul-torfya...,"Аренда здания на ул. ТорфянаяСормовский район,...",104 000 руб./мес.,Сдается в аренду холодный склад 650 кв.м. Мате...,2823434,Сормовский,ул. Торфяная,650 м²,1,металлический,отдельно стоящее здание,складское,NaN,gipernn,Комерческая недвижимость,Аренда зданий,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/arenda-zdaniy/ul-goroho...,"Аренда здания на ул. Гороховецкая, д. 7БКанави...",110 000 руб./мес.,"Красная линия! \nСдаю здание 150, 8 кв.м. с з...",2828846,Канавинский,"ул. Гороховецкая, д. 7Б",151 м²,1,NaN,отдельно стоящее здание,торговое,NaN,gipernn,Комерческая недвижимость,Аренда зданий,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/arenda-zdaniy/ul-arteln...,"Аренда здания на ул. Артельная, д. 1Советский ...",450 000 руб./мес.,"Советский район, улица Проспект Гагарина, Арте...",2842969,Советский,"ул. Артельная, д. 1",930 м²,3,Уточнить,отдельно стоящее здание,"офисное, образовательное учреждение",NaN,gipernn,Комерческая недвижимость,Аренда зданий,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('buildings-rent.xlsx', index=False)